インポート

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
from scipy import signal
import math
from numba import jit, i2
import datetime

テキストファイルの読み込み

In [ ]:
repeatUnitFileName='CUP1_repeatUnit.txt'
f = open(repeatUnitFileName, 'r')
repeatUnitList = f.readlines()
f.close()

In [ ]:
sequenceFileName='testSequence.txt'
f = open(sequenceFileName, 'r')
sequenceList = f.readlines()
f.close()

テキストファイルの作成

In [ ]:
dt_now = datetime.datetime.now()

with open('SD.txt', mode='a') as f:
    f.write('\nDNA_Sequence_Detector'+
            '\n\n開始時刻　'+dt_now.strftime('%Y-%m-%d %H:%M:%S')+
            '\n') 

In [ ]:
repeatUnitName=''
repeatUnit=''
for i in range(0,len(repeatUnitList)):   
    if( '>'in repeatUnitList[i] ):
        repeatUnitName=repeatUnitName+repeatUnitList[i][1:-1]
    else:
        repeatUnit=repeatUnit+repeatUnitList[i][0:-1]

with open('SD.txt', mode='a') as f:
    f.write('\n既知の配列のファイルの名前　'+repeatUnitFileName+
            '\n既知の配列の名前　'+repeatUnitName+
            '\n既知の配列の長さ　'+str( len(repeatUnit) )+
            '\n') 

In [ ]:
with open('SD.txt', mode='a') as f:
    f.write('\nSequenceファイルの名前　'+sequenceFileName) 

In [ ]:
nameCounter=0
for i in range(len(sequenceList)):
    if('>' in sequenceList[i]):
        nameCounter=nameCounter+1
with open('SD.txt', mode='a') as f:
    f.write('\nSequenceファイル中のSequenceの個数　'+str(nameCounter)+
            '\n') 

In [ ]:
#NF2決定
E1=0
for i in range(10+1, 1000):
    E1 = E1 + (9/16)*( (1/4)**i )*(i**2)

NF2List=[1]
for NF2 in NF2List:
    E2=0
    for i in range(NF2+1, 1000):
        E2 = E2 + (9/16)*( (1/4)**i )*(i**2)
    if(len(repeatUnit)*E2 <= 1988*E1):
        break
    else:
        NF2List.append(NF2+1)
NF2=NF2List[-1]

In [ ]:
#パラメータ決定
NF1=0
#NF2=10
L1 = np.round(len(repeatUnit)/20).astype(np.int)
density = (2*NF2+1+1+1)/L1
L1_R = np.round(len(repeatUnit)/4).astype(np.int)
L2 = np.round(len(repeatUnit)/4).astype(np.int)
minLength = np.round(len(repeatUnit)/2).astype(np.int)
Ratio=1.5
N_fourier=10

In [ ]:
with open('SD.txt', mode='a') as f:
    f.write('\nNF1='+str(NF1)+
            '\nNF2='+str(NF2)+
            '\nL1='+str(L1)+
            '\ndensity='+str(density)+
            '\nL1_R='+str(L1_R)+
            '\nL2='+str(L2)+
            '\nminLength='+str(minLength)+
            '\nRatio='+str(Ratio)+
            '\nN_fourier='+str(N_fourier)+
            '\n') 

In [ ]:
with open('SD.txt', mode='a') as f:
    f.write('\n元の配列の名前'+
            '\t配列の長さ'+
            '\t既知配列開始位置'+
            '\t既知配列終了位置'+
            '\t既知配列の個数(推定)'+
            '\t既知配列の個数(フーリエ変換)'+
            '\t第n候補'+
            '\t既知配列の長さ(フーリエ変換)'+
            '\t領域判定値')

In [ ]:
Index=0
for j in tqdm(range(nameCounter)):
    name=''
    seq=''
    counter=0
    for i in range(Index,len(sequenceList)):   
        if( ('>'in sequenceList[i]) and (counter==0) ):
            name=name+"'"+sequenceList[i][1:-1]
            counter=counter+1
        elif( ('>'in sequenceList[i]) and (counter>0) ):
            Index=i
            break
        else:
            seq=seq+sequenceList[i][0:-1]
    
    %run SD_internal.ipynb

dt_now = datetime.datetime.now()
with open('SD.txt', mode='a') as f:
    f.write('\n\n終了時刻　'+dt_now.strftime('%Y-%m-%d %H:%M:%S')+
            '\n') 